In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datayhatinh/y_hatinh.npy
/kaggle/input/dataxhatinh/x_hatinh.npy


In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    """
    Phiên bản nhẹ hơn cho tài nguyên hạn chế
    """

    def __init__(self, in_channels_sat=13, in_channels_met=8, base_filters=32):
        super(CNN, self).__init__()

        # Kết hợp input ngay từ đầu
        total_channels = in_channels_sat + in_channels_met

        self.encoder = nn.Sequential(
            # Block 1 - không dùng MaxPool, dùng stride=2
            nn.Conv2d(total_channels, base_filters, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm2d(base_filters),
            nn.ReLU(inplace=True),

            # Block 2 - không dùng MaxPool, dùng stride=2
            nn.Conv2d(base_filters, base_filters * 2, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(base_filters * 2),
            nn.ReLU(inplace=True),

            # Block 3 - giữ nguyên kích thước
            nn.Conv2d(base_filters * 2, base_filters * 4, kernel_size=3, padding=1),
            nn.BatchNorm2d(base_filters * 4),
            nn.ReLU(inplace=True),
            nn.Conv2d(base_filters * 4, base_filters * 4, kernel_size=3, padding=1),
            nn.BatchNorm2d(base_filters * 4),
            nn.ReLU(inplace=True)
        )

        self.decoder = nn.Sequential(
            nn.Conv2d(base_filters * 4, base_filters * 2, kernel_size=3, padding=1),
            nn.BatchNorm2d(base_filters * 2),
            nn.ReLU(inplace=True),

            nn.Conv2d(base_filters * 2, base_filters, kernel_size=3, padding=1),
            nn.BatchNorm2d(base_filters),
            nn.ReLU(inplace=True),

            # Output
            nn.Conv2d(base_filters, 1, kernel_size=1),
            nn.ReLU()
        )

    def forward(self, sat_data, met_data):
        # Bỏ dimension thừa do DataLoader tạo ra
        if sat_data.dim() == 5:
            sat_data = sat_data.squeeze(1)
        if met_data.dim() == 5:
            met_data = met_data.squeeze(1)

        # Lưu kích thước gốc
        original_size = (sat_data.shape[2], sat_data.shape[3])  # (21, 34)

        # Kết hợp inputs
        x = torch.cat([sat_data, met_data], dim=1)  # (batch, 34, 21, 34)

        # Encode
        encoded = self.encoder(x)  # (batch, 128, ~5, ~8)
        
        # Decode
        decoded = self.decoder(encoded)  # (batch, 1, ~5, ~8)
        
        # Upsample về kích thước gốc
        rainfall = F.interpolate(decoded, size=original_size, mode='bilinear', align_corners=False)
        rainfall = rainfall.squeeze(1)  # (batch, 21, 34)

        return rainfall

In [41]:
import torch
import torch.nn as nn


class RainfallLoss(nn.Module):
    """Custom loss cho rainfall estimation"""

    def __init__(self, alpha=0.7):
        super().__init__()
        self.alpha = alpha
        self.mse = nn.MSELoss()
        self.mae = nn.L1Loss()

    def forward(self, pred, target):
        return self.alpha * self.mse(pred, target) + (1 - self.alpha) * self.mae(pred, target)


def calculate_metrics(pred, target):
    """Tính RMSE, MAE, Correlation"""
    with torch.no_grad():
        mse = torch.mean((pred - target) ** 2)
        rmse = torch.sqrt(mse)
        mae = torch.mean(torch.abs(pred - target))

        pred_flat = pred.flatten()
        target_flat = target.flatten()

        # Correlation
        pred_mean = pred_flat.mean()
        target_mean = target_flat.mean()

        numerator = ((pred_flat - pred_mean) * (target_flat - target_mean)).sum()
        denominator = torch.sqrt(((pred_flat - pred_mean) ** 2).sum() *
                                 ((target_flat - target_mean) ** 2).sum())

        corr = numerator / (denominator + 1e-8)

    return {
        'RMSE': rmse.item(),
        'MAE': mae.item(),
        'Correlation': corr.item()
    }

In [42]:
# learning_rate_schedule.py
import math
import torch

class MyScheduler(torch.optim.lr_scheduler.LRScheduler):
    """
    Warmup theo ratio + step-based decay (linear, cosine, exponential)
    """
    def __init__(self, optimizer, total_steps, warmup_ratio=0.1,
                 scheduler_type="cosine", gamma=0.1, final_lr_ratio=0.0, last_epoch=-1):
        """
        Args:
            optimizer: optimizer
            total_steps: tổng step (batch*epoch)
            warmup_ratio: tỉ lệ warm-up (0~1)
            scheduler_type: "linear", "cosine", "exponential"
            gamma: dùng cho exponential decay
            final_lr_ratio: lr cuối / lr_max (linear/cosine)
        """
        self.scheduler_type = scheduler_type.lower()
        self.gamma = gamma
        self.warmup_steps = max(1, int(total_steps * warmup_ratio))
        self.total_steps = total_steps
        self.final_lr_ratio = final_lr_ratio
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        step = self.last_epoch + 1
        lrs = []
        for base_lr in self.base_lrs:
            if step <= self.warmup_steps:
                # linear warmup
                lr = base_lr * step / self.warmup_steps
            else:
                decay_step = step - self.warmup_steps
                decay_total = self.total_steps - self.warmup_steps
                if self.scheduler_type == "linear":
                    lr = base_lr - (base_lr - base_lr * self.final_lr_ratio) * decay_step / decay_total
                elif self.scheduler_type == "cosine":
                    lr = base_lr * 0.5 * (1 + math.cos(math.pi * decay_step / decay_total))
                    lr = max(lr, base_lr * self.final_lr_ratio)
                elif self.scheduler_type == "exponential":
                    lr = base_lr * (self.gamma ** decay_step)
                else:
                    raise ValueError(f"Scheduler type {self.scheduler_type} not supported")
            lrs.append(lr)
        return lrs

    def step(self, epoch=None):
        super().step(epoch)

# ===========================
# Example usage
# ===========================
# if __name__ == "__main__":
#     import torch.nn as nn
#     import torch.optim as optim

#     model = nn.Linear(10, 2)
#     optimizer = optim.Adam(model.parameters(), lr=0.001)

#     total_epochs = 10
#     steps_per_epoch = 5
#     total_steps = total_epochs * steps_per_epoch

#     # Step scheduler với 10% warm-up
#     scheduler = MyScheduler(optimizer, scheduler_type="step",
#                               total_steps=total_steps, warmup_ratio=0.1,
#                               step_size=3, gamma=0.5)

#     for step in range(total_steps):
#         print(f"Step {step+1}, LR: {optimizer.param_groups[0]['lr']:.6f}")
#         scheduler.step()

In [43]:
import numpy as np

# Đọc data
X = np.load('/kaggle/input/dataxhatinh/x_hatinh.npy')
Y = np.load('/kaggle/input/datayhatinh/y_hatinh.npy')

# Kiểm tra shape
print(f"X shape: {X.shape}")  # (884, 34, 21, 34)
print(f"Y shape: {Y.shape}")  # (884, 21, 34)
print(f"X dtype: {X.dtype}")
print(f"Y dtype: {Y.dtype}")

X shape: (884, 34, 21, 34)
Y shape: (884, 21, 34)
X dtype: float64
Y dtype: float64


In [44]:
import torch
from torch.utils.data import Dataset
import numpy as np
import os
from pathlib import Path
import glob


class RainfallDataset(Dataset):
    """
    Dataset khi dữ liệu đã load sẵn vào memory (numpy arrays)
    Dùng cho dataset nhỏ
    """

    def __init__(self, sat_data, met_data, rainfall_data,
                 transform=None, normalize=True):
        """
        Args:
            sat_data: numpy array (N, 13, H, W)
            met_data: numpy array (N, 8, H, W)
            rainfall_data: numpy array (N, 1, H, W) hoặc (N, H, W)
        """
        assert len(sat_data) == len(met_data) == len(rainfall_data), \
            "All arrays must have same length"

        self.sat_data = sat_data
        self.met_data = met_data

        # Ensure rainfall has shape (N, 1, H, W)
        if rainfall_data.ndim == 3:
            self.rainfall_data = rainfall_data[:, np.newaxis, :, :]
        else:
            self.rainfall_data = rainfall_data

        self.transform = transform
        self.normalize = normalize

        # Compute statistics
        if self.normalize:
            self.sat_mean = sat_data.mean(axis=(0, 2, 3), keepdims=True)
            self.sat_std = sat_data.std(axis=(0, 2, 3), keepdims=True)
            self.met_mean = met_data.mean(axis=(0, 2, 3), keepdims=True)
            self.met_std = met_data.std(axis=(0, 2, 3), keepdims=True)

        print(f"RainfallDataset initialized with {len(sat_data)} samples")

    def __len__(self):
        return len(self.sat_data)

    def __getitem__(self, idx):
        sat = self.sat_data[idx].copy()
        met = self.met_data[idx].copy()
        rainfall = self.rainfall_data[idx].copy()

        # Normalize
        if self.normalize:
            sat = (sat - self.sat_mean) / (self.sat_std + 1e-8)
            met = (met - self.met_mean) / (self.met_std + 1e-8)

        # Convert to tensors
        sat = torch.from_numpy(sat).float()
        met = torch.from_numpy(met).float()
        rainfall = torch.from_numpy(rainfall).float()

        # Apply transforms
        if self.transform:
            combined = torch.cat([sat, met, rainfall], dim=0)
            combined = self.transform(combined)
            sat = combined[:13]
            met = combined[13:21]
            rainfall = combined[21:]

        return sat, met, rainfall


In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt





class Trainer:
    def __init__(self, model, train_dataloader, val_dataloader=None,
                 criterion=None, optimizer=None, scheduler=None,
                 device='cuda', log_step=50, val_step=200,
                 model_save_path='best_model.pth'):
        self.model = model
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.log_step = log_step
        self.val_step = val_step
        self.model_save_path = model_save_path

        self.best_val_loss = float('inf')
        self.global_step = 0

        self.model.to(self.device)

        # Tổng số tham số
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

        print(f"Total parameters: {total_params:,}")
        print(f"Trainable parameters: {trainable_params:,}")

    def get_loss(self, batch):
        """Tính loss cho một batch"""
        sat, met, target = batch
        # DEBUG: In shape trước khi .to(device)
        print(f"DEBUG - sat shape before .to: {sat.shape}")
        print(f"DEBUG - met shape before .to: {met.shape}")
        print(f"DEBUG - target shape before .to: {target.shape}")

        sat = sat.to(self.device)
        met = met.to(self.device)
        target = target.to(self.device)

        # DEBUG: In shape sau khi .to(device)
        print(f"DEBUG - sat shape after .to: {sat.shape}")
        print(f"DEBUG - met shape after .to: {met.shape}")

        outputs = self.model(sat, met)
        loss = self.criterion(outputs, target)

        return loss

    def train(self, num_epochs=10, accumulate_steps=1, save_by='loss'):
        """
        Training loop với gradient accumulation
        accumulate_steps: số batch gom lại trước khi update optimizer
        save_by: 'loss', 'rmse', hoặc 'mae' - metric để save best model
        """
        total_batches = len(self.train_dataloader)
        total_updates = (total_batches + accumulate_steps - 1) // accumulate_steps  # ceil

        history = {'train_loss': [], 'val_loss': [], 'val_rmse': [], 'val_mae': [], 'val_corr': []}

        for epoch in range(num_epochs):
            self.model.train()
            running_loss = 0.0
            loop = tqdm(total=total_updates, desc=f"Epoch {epoch + 1}/{num_epochs}",
                        unit="it", ncols=100)
            update_count = 0

            for batch_idx, batch in enumerate(self.train_dataloader):
                self.global_step += 1

                loss = self.get_loss(batch)
                loss = loss / accumulate_steps  # chia loss cho accumulate_steps
                loss.backward()

                # Cập nhật optimizer sau mỗi accumulate_steps batch
                if (batch_idx + 1) % accumulate_steps == 0 or (batch_idx + 1) == total_batches:
                    # --- grad norm ---
                    total_norm = 0.0
                    for p in self.model.parameters():
                        if p.grad is not None:
                            total_norm += (p.grad.data.norm(2).item()) ** 2
                    total_norm = total_norm ** 0.5

                    self.optimizer.step()
                    self.optimizer.zero_grad()
                    if self.scheduler is not None:
                        self.scheduler.step()

                    running_loss += loss.item() * batch[0].size(0) * accumulate_steps  # nhân lại
                    update_count += 1
                    loop.update(1)  # update tqdm theo số update, không phải batch

                    # --- logging ---
                    if update_count % self.log_step == 0:
                        avg_loss = running_loss / ((batch_idx + 1) * self.train_dataloader.batch_size)
                        current_lr = self.optimizer.param_groups[0]['lr']
                        tqdm.write(f"Step {self.global_step}\tTrain Loss: {avg_loss:.7f}\t"
                                   f"Grad Norm: {total_norm:.7f}\tLR: {current_lr:.6f}")

                    # --- validation ---
                    if self.val_dataloader is not None and update_count % self.val_step == 0:
                        val_loss, metrics = self.validate_and_save(save_by=save_by)
                        tqdm.write(f"Val Loss: {val_loss:.4f} | RMSE: {metrics['RMSE']:.4f} | "
                                   f"MAE: {metrics['MAE']:.4f}")
                else:
                    running_loss += loss.item() * batch[0].size(0)

            # --- cuối epoch ---
            epoch_train_loss = running_loss / len(self.train_dataloader.dataset)
            history['train_loss'].append(epoch_train_loss)

            if self.val_dataloader is not None:
                epoch_val_loss, metrics = self.validate_and_save(save_by=save_by)
                history['val_loss'].append(epoch_val_loss)
                history['val_rmse'].append(metrics['RMSE'])
                history['val_mae'].append(metrics['MAE'])
                history['val_corr'].append(metrics['Correlation'])

                print(f"Epoch {epoch + 1}/{num_epochs} finished, "
                      f"Train Loss: {epoch_train_loss:.7f}, Val Loss: {epoch_val_loss:.7f}")
                print(f"  RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}, Corr: {metrics['Correlation']:.4f}")
            else:
                print(f"Epoch {epoch + 1}/{num_epochs} finished, Train Loss: {epoch_train_loss:.7f}")

            loop.close()

        return self.model, history

    def validate_and_save(self, save_by='loss'):
        """
        Validation và save model
        save_by: 'loss' (default), 'rmse', hoặc 'mae' - metric để quyết định best model
        """
        self.model.eval()

        total_loss = 0.0
        total_samples = 0
        all_metrics = {'RMSE': 0, 'MAE': 0, 'Correlation': 0}

        with torch.no_grad():
            for batch in tqdm(self.val_dataloader, desc="Validation", leave=False):
                loss = self.get_loss(batch)

                batch_size = batch[0].size(0)
                total_loss += loss.item() * batch_size
                total_samples += batch_size

                # Tính metrics
                sat, met, target = batch
                sat = sat.to(self.device)
                met = met.to(self.device)
                target = target.to(self.device)
                outputs = self.model(sat, met)

                metrics = calculate_metrics(outputs, target)
                for k, v in metrics.items():
                    all_metrics[k] += v * batch_size

        # loss mean trên dataset
        val_loss = total_loss / total_samples
        for k in all_metrics:
            all_metrics[k] /= total_samples

        # ----- Save theo best metric -----
        # Chọn metric để save
        if save_by == 'loss':
            current_metric = val_loss
            is_better = current_metric < self.best_val_loss
            metric_name = 'val_loss'
        elif save_by == 'rmse':
            current_metric = all_metrics['RMSE']
            is_better = current_metric < self.best_val_loss
            metric_name = 'RMSE'
        elif save_by == 'mae':
            current_metric = all_metrics['MAE']
            is_better = current_metric < self.best_val_loss
            metric_name = 'MAE'
        else:
            current_metric = val_loss
            is_better = current_metric < self.best_val_loss
            metric_name = 'val_loss'

        if is_better:
            self.best_val_loss = current_metric
            torch.save({
                'model_state_dict': self.model.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
                'val_loss': val_loss,
                'metrics': all_metrics,
                'epoch': self.global_step
            }, self.model_save_path)
            tqdm.write(f"Step {self.global_step}: New best model saved with {metric_name} {current_metric:.6f}")
            tqdm.write(f"  Metrics - RMSE: {all_metrics['RMSE']:.4f}, MAE: {all_metrics['MAE']:.4f}, "
                       f"Corr: {all_metrics['Correlation']:.4f}")

        self.model.train()
        return val_loss, all_metrics

if __name__ == "__main__":
    from torch.utils.data import random_split

    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    GRADIENT_ACCUMULATION_STEPS = 4
    VAL_BATCH_SIZE = 32
    BATCH_SIZE = 8
    VAL_RATIO = 0.2
    EPOCHS = 10
    log_step = 50
    val_step = 200
    learning_rate = 1e-4
    weight_decay = 1e-5
    model_save_path = 'best_model.pth'
    SCHEDULER_TYPE = 'cosine'
    WARMUP_RATIO = 0.1
    GAMMA = 0.95

    sat_data = X[:, :20, :, :]   # 20 channels đầu
    met_data = X[:, 20:, :, :]   # 14 channels sau
    rainfall_data = Y

    model = CNN(in_channels_sat=20, in_channels_met=14, base_filters=32)    
    criterion = RainfallLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    
    dataset = RainfallDataset(
            sat_data=sat_data,
            met_data=met_data,
            rainfall_data=rainfall_data,
            transform=None,
            normalize=True)

    N = len(dataset)
    train_size = int(N * (1 - VAL_RATIO))

    # Index tách theo thứ tự không random
    indices = list(range(N))

    train_idx = indices[:train_size]
    val_idx = indices[train_size:]

    train_dataset = torch.utils.data.Subset(dataset, train_idx)
    val_dataset = torch.utils.data.Subset(dataset, val_idx)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    # Tính total updates cho scheduler
    total_updates = (len(train_dataset) + GRADIENT_ACCUMULATION_STEPS - 1) // GRADIENT_ACCUMULATION_STEPS * EPOCHS

    scheduler = MyScheduler(
        optimizer,
        total_steps=total_updates,
        scheduler_type=SCHEDULER_TYPE,
        warmup_ratio=WARMUP_RATIO,
        gamma=GAMMA
    )

    # Khởi tạo trainer
    trainer = Trainer(
        model,
        train_dataloader=train_loader,
        val_dataloader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=DEVICE,
        log_step=log_step,
        val_step=val_step,
        model_save_path=model_save_path
    )

    # Train model
    trained_model, history = trainer.train(
        num_epochs=EPOCHS,
        accumulate_steps=GRADIENT_ACCUMULATION_STEPS,
        save_by='rmse'  # Có thể chọn 'loss', 'rmse', hoặc 'mae'
    )


RainfallDataset initialized with 884 samples
Total parameters: 360,353
Trainable parameters: 360,353






Epoch 1/10:   0%|                                                            | 0/23 [00:00<?, ?it/s]

DEBUG - sat shape before .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape before .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - target shape before .to: torch.Size([8, 1, 21, 34])
DEBUG - sat shape after .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape after .to: torch.Size([8, 1, 14, 21, 34])


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([8, 1, 21, 34])) that is different to the input size (torch.Size([8, 21, 34])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([8, 1, 21, 34])) that is different to the input size (torch.Size([8, 21, 34])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)




Epoch 1/10:   4%|██▎                                                 | 1/23 [00:00<00:11,  1.99it/s]

DEBUG - sat shape before .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape before .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - target shape before .to: torch.Size([8, 1, 21, 34])
DEBUG - sat shape after .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape after .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - sat shape before .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape before .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - target shape before .to: torch.Size([8, 1, 21, 34])
DEBUG - sat shape after .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape after .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - sat shape before .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape before .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - target shape before .to: torch.Size([8, 1, 21, 34])
DEBUG - sat shape after .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape after .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - sat shape before .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape before .to: tor

...

DEBUG - sat shape before .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape before .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - target shape before .to: torch.Size([8, 1, 21, 34])
DEBUG - sat shape after .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape after .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - sat shape before .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape before .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - target shape before .to: torch.Size([8, 1, 21, 34])
DEBUG - sat shape after .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape after .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - sat shape before .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape before .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - target shape before .to: torch.Size([8, 1, 21, 34])
DEBUG - sat shape after .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape after .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - sat shape before .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape before .to: tor






Validation:   0%|          | 0/23 [00:00<?, ?it/s]

DEBUG - sat shape before .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape before .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - target shape before .to: torch.Size([8, 1, 21, 34])
DEBUG - sat shape after .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape after .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - sat shape before .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape before .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - target shape before .to: torch.Size([8, 1, 21, 34])
DEBUG - sat shape after .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape after .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - sat shape before .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape before .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - target shape before .to: torch.Size([8, 1, 21, 34])
DEBUG - sat shape after .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape after .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - sat shape before .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape before .to: tor






Validation:  96%|█████████▌| 22/23 [00:00<00:00, 215.53it/s]




Epoch 10/10: 100%|██████████████████████████████████████████████████| 23/23 [00:00<00:00, 35.56it/s]

DEBUG - sat shape before .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape before .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - target shape before .to: torch.Size([8, 1, 21, 34])
DEBUG - sat shape after .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape after .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - sat shape before .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape before .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - target shape before .to: torch.Size([8, 1, 21, 34])
DEBUG - sat shape after .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape after .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - sat shape before .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape before .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - target shape before .to: torch.Size([8, 1, 21, 34])
DEBUG - sat shape after .to: torch.Size([8, 1, 20, 21, 34])
DEBUG - met shape after .to: torch.Size([8, 1, 14, 21, 34])
DEBUG - sat shape before .to: torch.Size([1, 1, 20, 21, 34])
DEBUG - met shape before .to: tor